In [1]:
from pynq import Overlay
from pynq import MMIO
from time import time

In [2]:
# Program bitstream to FPGA
overlay = Overlay('/home/ubuntu/workspace/axi_lite_gcd.bit')

In [3]:
# Access to memory map of the AXI GCD
ADDR_BASE = 0xA0000000
ADDR_RANGE = 0x80
gcd_obj = MMIO(ADDR_BASE, ADDR_RANGE)

In [4]:
# Write input A and B to the AXI GCD
gcd_obj.write(0x8, 128)
gcd_obj.write(0x10, 72)

In [5]:
# Start main controller
gcd_obj.write(0x0, 0x1)
# Wait until ready flag is 1
while ((gcd_obj.read(0x0) & (1 << 1)) == 0):
    pass

In [6]:
# Read GCD result
gcd_obj.read(0x18)

8

In [7]:
# Function to calculate GCD with HW core
def calc_gcd_hw(a, b):
    gcd_obj.write(0x8, a)
    gcd_obj.write(0x10, b)
    gcd_obj.write(0x0, 0x1)
    r = gcd_obj.read(0x18)
    return r

In [8]:
calc_gcd_hw(128, 72)

8

In [9]:
calc_gcd_hw(1680888, 8880168)

264

In [10]:
# Measure the time required to calculate 1 million operation
t1 = time()
for i in range(1000000):
    calc_gcd_hw(2391065, 3578129)
t2 = time()
t_hw = t2 - t1
print('Time used for HW GCD: {}s'.format(t_hw))

Time used for HW GCD: 29.42424488067627s


In [11]:
# Function to calculate GCD with SW
def calc_gcd_sw(a, b):
    n = 0
    while True:
        if (a == b):
            break
        if ((a % 2) == 0): # a even
            a = a >> 1
            if ((b % 2) == 0): # b even
                b = b >> 1
                n = n + 1
        else: # a odd
            if ((b % 2) == 0): # b even
                b = b >> 1
            else: # b odd
                if (a > b):
                    a = a - b
                else:
                    b = b - a
    a = a << n
    return a

In [12]:
calc_gcd_sw(128, 72)

8

In [13]:
calc_gcd_sw(1680888, 8880168)

264

In [14]:
# Measure the time required to calculate 1 million operation
t1 = time()
for i in range(1000000):
    calc_gcd_sw(2391065, 3578129)
t2 = time()
t_sw = t2 - t1
print('Time used for SW GCD: {}s'.format(t_sw))

Time used for SW GCD: 54.3738739490509s
